Deep Learning
=============

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import math
import tarfile, itertools
import pandas as pd
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, SGD
import tensorflow as tf
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import pdb

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

/Users/chientm/miniconda3/envs/subaru/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/chientm/miniconda3/envs/subaru/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/chientm/miniconda3/envs/subaru/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/chientm/miniconda3/envs/subaru/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/chientm/miniconda3/envs/subaru/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got

AttributeError: module 'tensorflow.python.training.checkpointable' has no attribute 'CheckpointableBase'

In [ ]:
pkl_files = 'full_data.pickle'
with open(pkl_files, 'rb') as file:
    data = pickle.load(file)
    print(data['train_dataset'].shape)
    print(data['train_labels'].shape)
    print(data['valid_dataset'].shape)
    print(data['valid_labels'].shape)
    print(data['test_dataset'].shape)
    print(data['test_labels'].shape)

---
Display
---------

Displaying some sample of the labels and images in training set.

---

In [ ]:
training_data = data['train_dataset']
training_label = data['train_labels']

letter_samples = []
indices = np.random.choice(training_data.shape[0], 10, replace=False)
for idx in indices:
    letter_samples.append(training_data[idx])

fig, ax = plt.subplots(2, 5, figsize=(5, 5))
for i, j in itertools.product(range(2), range(5)):
    ax[i, j].get_xaxis().set_visible(False)
    ax[i, j].get_yaxis().set_visible(False)
    ax[i, j].cla()
    ax[i, j].imshow(letter_samples[i*5 + j], cmap='gray')

plt.show()        

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [ ]:
def count_overlap(set1, set2):
    count = 0
    for img1 in set1:
        for img2 in set2:
            if np.array_equal(img1, img2):
                count += 1
    return count

train_valid = count_overlap(data['train_dataset'], data['valid_dataset'])
train_test = count_overlap(data['train_dataset'], data['test_dataset'])
valid_test = count_overlap(data['valid_dataset'], data['test_dataset'])

In [ ]:
print('Overlap between train and valid:', train_valid) # 37135
print('Overlap between train and test:', train_test) # 9991
print('Overlap between valid and test:', valid_test) # 1644

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

### Logistic regression model

In [ ]:
# Logistic regression model
nsamples, nx, ny = data['train_dataset'].shape
train_dataset = data['train_dataset'].reshape((nsamples, nx*ny))

lr_model = LogisticRegression()
lr_model.fit(train_dataset, data['train_labels'])

In [ ]:
nsamples, nx, ny = data['test_dataset'].shape
test_dataset = data['test_dataset'].reshape((nsamples, nx*ny))

acc = lr_model.score(test_dataset, data['test_labels'])
print('Accuracy: ', acc)

### TensorFlow Convolution model

In [ ]:
def next_batch(step, batch_size, data, labels):
    return data[step*batch_size:(step + 1)*batch_size], labels[step*batch_size:(step + 1)*batch_size]

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
# Input
x_input = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_label = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x_input, [-1, 28, 28, 1])

# Convoution layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convoution layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# FC layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Softmax layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

epochs = 30
batch_size = 100
steps = math.ceil(data['train_dataset'].shape[0] // batch_size)

y_train = pd.get_dummies(data['train_labels']).values
val_y = pd.get_dummies(data['valid_labels']).values
test_y = pd.get_dummies(data['test_labels']).values
train_accs = []
eval_accs = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for step in range(steps):
            batch_X, batch_y = next_batch(step, batch_size, data['train_dataset'], y_train)
            #if step % 50 == 0:
            #    train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 1.0})
            #    print('step %d, training accuracy %g' % (step, train_accuracy))
            train_step.run(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 0.5})
        train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 1.0})
        eval_accuracy = accuracy.eval(feed_dict={x_input: data['valid_dataset'], y_label: val_y, keep_prob: 1.0})
        train_accs.append(train_accuracy)
        eval_accs.append(eval_accuracy)
        print('Epoch: %d - Accuracy: %g' % (epoch, eval_accuracy))
    accuracy_l = accuracy.eval(feed_dict={x_input: data['test_dataset'], y_label: test_y, keep_prob: 1.0})
    print('test accuracy ', accuracy_l)
        
x = np.array(range(epochs))
plt.plot(x, train_accs)
plt.plot(x, eval_accs)
plt.show()

## Keras model

In [ ]:
model = Sequential()
# model.add(Reshape((28, 28, 1), input_shape=(784,)))
model.add(Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
epochs = 20
batch_size = 100
X_tr = np.expand_dims(data['train_dataset'], axis=len(data['train_dataset'].shape))
X_val = np.expand_dims(data['valid_dataset'], axis=len(data['valid_dataset'].shape))
model.fit(X_tr, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, val_y)
         )

In [ ]:
X_test = np.expand_dims(data['test_dataset'], axis=len(data['test_dataset'].shape))
score = model.evaluate(X_test, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])